In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import schedule, time

In [2]:
"""
    Only works for Firefox 45.0 version(Not 48.0)
    
    This program checks your subject on sugang.snu.ac.kr if there is any seat left.
    If there is seat left, it will automatically log-in to your gmail account and 
    send an email. Check it with your linked phone gmail application to get the updates ASAP.
    
    -Execute the program on 07:00 or before (will end at 16:00) (Do not turn off the computer)
    -If Handling error occurs(will not, probably) try restarting the kernel
    -If you want to search subjects differently(ex.subject code),
        you can search subject by subject code by differentiating the *elem* variable    
    -If sugang.snu.ac.kr html changes, you will have to inspect all_td in order to find the
        index where registered student number is. It's all_td[44::18] for now. Or just use the CSS_selector
"""

def sugang_check(subject, sId, sPassword):

    def send_email(number):    #subject='수업', sId=None, sPassword=None):
        driver2 = webdriver.Firefox()
        driver2.get("http://mail.google.com")
        name = driver2.find_element_by_name("Email")
        name.clear()
        name.send_keys(sId)
        submit = driver2.find_element_by_id("next")
        submit.click()

        wait = WebDriverWait(driver2, 1).until(
                EC.presence_of_element_located((By.ID, "Passwd"))
            )

        passwd = driver2.find_element_by_name('Passwd')
        passwd.clear()
        passwd.send_keys(sPassword+Keys.ENTER)

        #time.sleep(num_of_time)
        #driver.implicitly_wait(num_of_time)

        wait2 = WebDriverWait(driver2, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".T-I-KE"))
                            #dom f12 에서 원하는 element 오른쪽 클릭하고 unique selector로 받아서 찾는거임.
        )
        #Exception?
        
        compose = driver2.find_element_by_css_selector(".T-I-KE")
        compose.click()

        wati3 = WebDriverWait(driver2, 5).until(
                EC.presence_of_element_located((By.NAME, "to"))
        )
    #     driver.implicitly_wait(3)

        driver2.find_element_by_name("to").send_keys("jeshwang92@gmail.com")
        driver2.find_element_by_name('subjectbox').send_keys('알림'+Keys.TAB+'{0} 자리났음, {1}번째 수업.'.format(subject, number)\
                                                            +Keys.CONTROL+Keys.ENTER)
        driver2.quit()
        
        return 0

    driver = webdriver.Firefox()
    driver.get("http://sugang.snu.ac.kr/sugang/cc/cc100.action") #강좌검색 교과목명

    elem = driver.find_element_by_name("srchSbjtNm")
    elem.clear()
    elem.send_keys(subject)

    driver.find_element_by_class_name("btn_search_ok").click()

    all_td = driver.find_elements_by_tag_name('td')
    total = int(all_td[43].text[0:3]) #수강신청 정원 (첫 번째로 나온 과목 수강신청 정원 str 처음세자리 짜른거)

    #수강신청 인원이 44에서 18개씩 간격으로 나열되므로,.. all_td[44::18][i=0,1,..]로 웹사이트에 나열 순서된대로 수강신청인원 다 뽑아낼 수 있음
    sugangs = all_td[44::18]
    #검색된 모든 수업에 대하여 하는 방법
    for i, sugang in enumerate(sugangs, start=1):
        if int(sugang.text) < int(all_td[43+(i-1)*18].text[0:3]):
            send_email(i)
            time.sleep(1)
        else:
#             send_email(i)
            time.sleep(1)
            pass

    #특정 수업만 골라서 하는방법
#     if int(sugangs[0].text) < total or int(sugangs[1].text) < total:
#         send_email('0번, 1번 중 하나')
#     else:
#         pass
    #다른 sugang_check를 사용하고 싶으면, 몇 번째 수업을 하고 싶은지 넣어야함.
    
    driver.quit()

    return 0

In [3]:
#수강 도우미 돌아가는지 확인할 때- sugang_check(subject, sId, sPassword)

if __name__ == '__main__':

    subject = '호신술'
    sId = 'jeshwang92'
    sPassword = '90acd093!'

    subject2 = '댄스스포츠'

    schedule.every(2).minutes.do(sugang_check, subject, sId, sPassword)
    schedule.every(1).minutes.do(sugang_check, subject2, sId, sPassword)
    while True:
        schedule.run_pending()
    #     time.sleep(1)
        if schedule.time.strptime(schedule.time.asctime()).tm_hour == 16:
            schedule.clear()
            print('수강신청시간지남')
            break


KeyboardInterrupt: 

In [7]:
# subject = '호신술'
# sId = ''
# sPassword = ''
# sugang_check(subject, sId, sPassword)


#schedule.time.strptime('Tue Aug  9 16:00:00 2016')